In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
import warnings

warnings.filterwarnings(action = 'ignore')

In [50]:
df = pd.read_csv('../data/final-sample-fix.csv')
# pd.set_option('display.max_columns'19)
del(df['BIKE_NUM'])
del(df['CCTV_NUM'])

In [51]:
df

,Unnamed: 0,GU,DONG_CODE,DONG,SUBWAY_NUM,STARBUCKS_NUM,SAFE_DLVR_NUM,MC_NUM,KINDER_NUM,KIDS_NUM,GYM_NUM,GOLF_NUM,ACADEMY_NUM,ANIMAL_HSPT_NUM,CAFE,CAR_SHR_NUM,CHILD_MED_NUM,CON_NUM,DPTM_NUM
0,0,종로구,1111010100,청운동,0,0,0,0,0,0,0,0,2,1,8,0,0,1,0
1,1,종로구,1111010200,신교동,0,0,0,0,0,0,0,0,5,1,1,0,0,0,0
2,2,종로구,1111010300,궁정동,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0
3,3,종로구,1111010400,효자동,0,0,0,0,0,0,0,0,0,1,5,0,0,0,0
4,4,종로구,1111010500,창성동,0,0,0,0,0,0,0,0,1,1,3,0,2,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
462,462,강동구,1174010700,암사동,1,1,2,0,30,4,16,8,197,1,70,4,11,33,0
463,463,강동구,1174010900,천호동,1,4,3,1,70,6,14,7,124,1,131,17,12,68,1
464,464,강동구,1174010800,성내동,4,4,2,0,50,2,30,14,190,1,143,20,12,58,0
465,465,강동구,1174010500,길동,0,2,1,0,20,1,18,9,100,1,63,10,10,38,0


In [53]:
df.info()
# [구분]
# 0 : index
# 1 : 구 이름
# 2 : 동 코드
# 3 : 동 이름

# [교통]
# 4 : 지하철
#14 : 따릉이 - 데이터 누락

# [방범]
# 6 : 안심택배
#17 : CCTV - 데이터 누락

# [건강] : 병원, 약국 정보
#18 : 소아과

# [편의]
#15 : 카페
#19 : 편의점
#20 : 백화점

# [2차 분류]
# 5 : 스타벅스
# 7 : 맥도날드
    # [육아]
    # 8 : 유치원
    # 9 : 키즈카페
#10 : 체육시설
#11 : 골프장
#12 : 학원
#13 : 동물병원
#16 : 카쉐어링


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 467 entries, 0 to 466
Data columns (total 19 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Unnamed: 0       467 non-null    int64 
 1   GU               467 non-null    object
 2   DONG_CODE        467 non-null    int64 
 3   DONG             467 non-null    object
 4   SUBWAY_NUM       467 non-null    int64 
 5   STARBUCKS_NUM    467 non-null    int64 
 6   SAFE_DLVR_NUM    467 non-null    int64 
 7   MC_NUM           467 non-null    int64 
 8   KINDER_NUM       467 non-null    int64 
 9   KIDS_NUM         467 non-null    int64 
 10  GYM_NUM          467 non-null    int64 
 11  GOLF_NUM         467 non-null    int64 
 12  ACADEMY_NUM      467 non-null    int64 
 13  ANIMAL_HSPT_NUM  467 non-null    int64 
 14  CAFE             467 non-null    int64 
 15  CAR_SHR_NUM      467 non-null    int64 
 16  CHILD_MED_NUM    467 non-null    int64 
 17  CON_NUM          467 non-null    in

In [54]:
df.rename(columns={'Unnamed: 0':'index'},inplace=True)
df.set_index('index',inplace=True)

In [78]:
df.describe()

,DONG_CODE,SUBWAY_NUM,STARBUCKS_NUM,SAFE_DLVR_NUM,MC_NUM,KINDER_NUM,KIDS_NUM,GYM_NUM,GOLF_NUM,ACADEMY_NUM,ANIMAL_HSPT_NUM,CAFE,CAR_SHR_NUM,CHILD_MED_NUM,CON_NUM,DPTM_NUM
count,4.670000e+02,467.000000,467.000000,467.000000,467.000000,467.000000,467.000000,467.000000,467.000000,467.000000,467.0,467.000000,467.000000,467.000000,467.000000,467.000000
mean,1.131524e+09,0.554604,1.201285,0.567452,0.186296,19.721627,0.473233,6.794433,3.310493,52.668094,1.0,34.650964,3.770878,13.066381,12.451820,0.057816
std,1.992829e+07,1.105349,2.525016,1.234887,0.483177,38.257761,1.106376,13.548535,7.546135,124.718890,0.0,56.393393,6.340226,27.919986,21.510436,0.267876
min,1.111010e+09,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.114013e+09,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.0,5.000000,0.000000,0.000000,1.000000,0.000000
50%,1.126010e+09,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,7.000000,1.0,14.000000,1.000000,3.000000,3.000000,0.000000
75%,1.150010e+09,1.000000,1.000000,1.000000,0.000000,20.000000,0.000000,7.000000,3.000000,57.500000,1.0,38.000000,5.000000,12.000000,14.000000,0.000000
max,1.174011e+09,7.000000,25.000000,10.000000,4.000000,220.000000,7.000000,120.000000,67.000000,1620.000000,1.0,503.000000,40.000000,256.000000,162.000000,3.000000


In [56]:
col = df.columns[3:]
data = df[col].values

In [44]:
df[df['GU'] == '종로구']['DONG'].unique()

array(['청운동', '신교동', '궁정동', '효자동', '창성동', '통인동', '누상동', '누하동', '옥인동',
       '세종로', '통의동', '적선동', '체부동', '필운동', '내자동', '사직동', '도렴동', '당주동',
       '내수동', '신문로1가', '신문로2가', '팔판동', '삼청동', '안국동', '소격동', '화동', '사간동',
       '송현동', '부암동', '홍지동', '신영동', '구기동', '평창동', '무악동', '교남동', '평동',
       '송월동', '홍파동', '교북동', '행촌동', '가회동', '재동', '계동', '원서동', '청진동', '서린동',
       '수송동', '중학동', '종로1가', '공평동', '관훈동', '견지동', '와룡동', '권농동', '운니동',
       '익선동', '경운동', '관철동', '인사동', '낙원동', '종로2가', '훈정동', '묘동', '봉익동',
       '돈의동', '장사동', '관수동', '종로3가', '인의동', '예지동', '원남동', '종로4가', '연지동',
       '효제동', '종로5가', '종로6가', '충신동', '이화동', '연건동', '동숭동', '혜화동', '명륜1가',
       '명륜2가', '명륜4가', '명륜3가', '창신동', '숭인동'], dtype=object)

In [36]:
dong_df = pd.read_excel('../data/동별_데이터.xlsx')
dong_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 450 entries, 0 to 449
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   GU          450 non-null    object 
 1   DONG        450 non-null    object 
 2   인구 (명)      450 non-null    int64  
 3   면적 (㎢)      450 non-null    float64
 4   인구밀도 (명/㎢)  450 non-null    int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 17.7+ KB


In [45]:
dong_df['GU'].unique()

array(['종로구', '중구', '용산구', '성동구', '동대문구', '중랑구', '성북구', '강북구', '도봉구',
       '노원구', '은평구', '서대문구', '마포구', '양천구', '강서구', '구로구', '영등포구', '동작구',
       '관악구', '강남구', '송파구', '강동구'], dtype=object)

In [29]:
df['GU'].unique() # 광진구, 금천구, 서초구

array(['종로구', '중구', '용산구', '성동구', '광진구', '동대문구', '중랑구', '성북구', '강북구',
       '도봉구', '노원구', '은평구', '서대문구', '마포구', '양천구', '강서구', '구로구', '금천구',
       '영등포구', '동작구', '관악구', '서초구', '강남구', '송파구', '강동구'], dtype=object)

In [38]:
dong_df[dong_df['GU'] == '종로구']

,GU,DONG,인구 (명),면적 (㎢),인구밀도 (명/㎢)
0,종로구,사직동,9636,1.23,7834
1,종로구,삼청동,2739,1.49,1838
2,종로구,부암동,9782,2.27,4309
3,종로구,평창동,18329,8.87,2066
4,종로구,무악동,8297,0.36,23047
5,종로구,교남동,10203,0.35,29151
6,종로구,가회동,4194,0.54,7767
7,종로구,종로1.2.3.4가동,8595,2.35,3657
8,종로구,종로5.6가동,5476,0.60,9127
9,종로구,이화동,7705,0.78,9878


In [41]:
df[df['DONG'] == '명륜1가']

,GU,DONG_CODE,DONG,SUBWAY_NUM,STARBUCKS_NUM,SAFE_DLVR_NUM,MC_NUM,KINDER_NUM,KIDS_NUM,GYM_NUM,GOLF_NUM,ACADEMY_NUM,ANIMAL_HSPT_NUM,BIKE_NUM,CAFE,CAR_SHR_NUM,CCTV_NUM,CHILD_MED_NUM,CON_NUM,DPTM_NUM
index,,,,,,,,,,,,,,,,,,,,
81,종로구,1111017000,명륜1가,0,0,0,0,0,0,0,0,15,1,0,4,0,0,7,1,0


In [43]:
dong_df[dong_df['DONG'] == '명륜1가']

,GU,DONG,인구 (명),면적 (㎢),인구밀도 (명/㎢)


In [14]:
kmeans = KMeans(n_clusters=10, init='k-means++',max_iter=300,random_state=0)
kmeans.fit(data)

KMeans(n_clusters=10, random_state=0)

In [15]:
kmeans.labels_
tmp_df = df.copy()
tmp_df['km_cluster'] = kmeans.labels_
tmp_df.groupby('km_cluster').count()

,GU,DONG_CODE,DONG,SUBWAY_NUM,STARBUCKS_NUM,SAFE_DLVR_NUM,MC_NUM,KINDER_NUM,KIDS_NUM,GYM_NUM,GOLF_NUM,ACADEMY_NUM,ANIMAL_HSPT_NUM,BIKE_NUM,CAFE,CAR_SHR_NUM,CCTV_NUM,CHILD_MED_NUM,CON_NUM,DPTM_NUM
km_cluster,,,,,,,,,,,,,,,,,,,,
0,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60
1,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6
2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
3,315,315,315,315,315,315,315,315,315,315,315,315,315,315,315,315,315,315,315,315
4,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6
5,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
6,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19
7,42,42,42,42,42,42,42,42,42,42,42,42,42,42,42,42,42,42,42,42
8,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6


In [77]:
tmp_df.describe()

,DONG_CODE,SUBWAY_NUM,STARBUCKS_NUM,SAFE_DLVR_NUM,MC_NUM,KINDER_NUM,KIDS_NUM,GYM_NUM,GOLF_NUM,ACADEMY_NUM,ANIMAL_HSPT_NUM,CAFE,CAR_SHR_NUM,CHILD_MED_NUM,CON_NUM,DPTM_NUM,km_cluster
count,4.670000e+02,467.000000,467.000000,467.000000,467.000000,467.000000,467.000000,467.000000,467.000000,467.000000,467.0,467.000000,467.000000,467.000000,467.000000,467.000000,467.000000
mean,1.131524e+09,0.554604,1.201285,0.567452,0.186296,19.721627,0.473233,6.794433,3.310493,52.668094,1.0,34.650964,3.770878,13.066381,12.451820,0.057816,3.282655
std,1.992829e+07,1.105349,2.525016,1.234887,0.483177,38.257761,1.106376,13.548535,7.546135,124.718890,0.0,56.393393,6.340226,27.919986,21.510436,0.267876,2.016397
min,1.111010e+09,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.114013e+09,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.0,5.000000,0.000000,0.000000,1.000000,0.000000,3.000000
50%,1.126010e+09,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,7.000000,1.0,14.000000,1.000000,3.000000,3.000000,0.000000,3.000000
75%,1.150010e+09,1.000000,1.000000,1.000000,0.000000,20.000000,0.000000,7.000000,3.000000,57.500000,1.0,38.000000,5.000000,12.000000,14.000000,0.000000,3.000000
max,1.174011e+09,7.000000,25.000000,10.000000,4.000000,220.000000,7.000000,120.000000,67.000000,1620.000000,1.0,503.000000,40.000000,256.000000,162.000000,3.000000,9.000000


In [19]:
tmp_df[tmp_df['km_cluster']==3]['DONG']

index
0       청운동
1       신교동
2       궁정동
3       효자동
4       창성동
       ... 
428     내곡동
429     염곡동
430     신원동
432    압구정동
443     율현동
Name: DONG, Length: 315, dtype: object

In [47]:
why = tmp_df[tmp_df['km_cluster']==3]
why

,GU,DONG_CODE,DONG,SUBWAY_NUM,STARBUCKS_NUM,SAFE_DLVR_NUM,MC_NUM,KINDER_NUM,KIDS_NUM,GYM_NUM,GOLF_NUM,ACADEMY_NUM,ANIMAL_HSPT_NUM,BIKE_NUM,CAFE,CAR_SHR_NUM,CCTV_NUM,CHILD_MED_NUM,CON_NUM,DPTM_NUM,km_cluster
index,,,,,,,,,,,,,,,,,,,,,
0,종로구,1111010100,청운동,0,0,0,0,0,0,0,0,2,1,0,8,0,0,0,1,0,3
1,종로구,1111010200,신교동,0,0,0,0,0,0,0,0,5,1,0,1,0,0,0,0,0,3
2,종로구,1111010300,궁정동,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,3
3,종로구,1111010400,효자동,0,0,0,0,0,0,0,0,0,1,0,5,0,0,0,0,0,3
4,종로구,1111010500,창성동,0,0,0,0,0,0,0,0,1,1,0,3,0,0,2,1,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
428,서초구,1165010900,내곡동,0,0,0,0,20,0,0,0,18,1,0,6,0,0,11,1,0,3
429,서초구,1165011000,염곡동,0,0,0,0,0,0,0,0,1,1,0,6,0,0,3,0,0,3
430,서초구,1165011100,신원동,0,1,0,0,10,0,0,0,7,1,0,16,0,0,8,6,0,3


In [48]:
why.describe()

,DONG_CODE,SUBWAY_NUM,STARBUCKS_NUM,SAFE_DLVR_NUM,MC_NUM,KINDER_NUM,KIDS_NUM,GYM_NUM,GOLF_NUM,ACADEMY_NUM,ANIMAL_HSPT_NUM,BIKE_NUM,CAFE,CAR_SHR_NUM,CCTV_NUM,CHILD_MED_NUM,CON_NUM,DPTM_NUM,km_cluster
count,3.150000e+02,315.000000,315.000000,315.000000,315.00000,315.000000,315.000000,315.000000,315.000000,315.000000,315.0,315.0,315.000000,315.000000,315.0,315.000000,315.000000,315.000000,315.0
mean,1.124146e+09,0.187302,0.469841,0.114286,0.04127,1.809524,0.060317,1.069841,0.438095,5.574603,1.0,0.0,10.695238,0.923810,0.0,3.079365,2.746032,0.028571,3.0
std,1.646594e+07,0.529238,0.849247,0.365231,0.19923,5.005911,0.318506,1.684004,0.877066,7.867331,0.0,0.0,9.988328,1.421145,0.0,5.053540,3.305893,0.216684,0.0
min,1.111010e+09,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,3.0
25%,1.111018e+09,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.0,4.000000,0.000000,0.0,0.000000,0.000000,0.000000,3.0
50%,1.114017e+09,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,2.000000,1.0,0.0,8.000000,0.000000,0.0,1.000000,2.000000,0.000000,3.0
75%,1.129013e+09,0.000000,1.000000,0.000000,0.00000,0.000000,0.000000,1.000000,1.000000,8.000000,1.0,0.0,15.000000,1.000000,0.0,4.000000,3.500000,0.000000,3.0
max,1.168011e+09,3.000000,5.000000,2.000000,1.00000,40.000000,4.000000,11.000000,5.000000,43.000000,1.0,0.0,61.000000,7.000000,0.0,29.000000,23.000000,3.000000,3.0


In [ ]:
why['']

In [57]:
kmeans = KMeans(n_clusters=10, init='k-means++',max_iter=300,random_state=0)
kmeans.fit(data)

KMeans(n_clusters=10, random_state=0)

In [58]:
kmeans.labels_
tmp_df = df.copy()
tmp_df['km_cluster'] = kmeans.labels_
tmp_df.groupby('km_cluster').count()

,GU,DONG_CODE,DONG,SUBWAY_NUM,STARBUCKS_NUM,SAFE_DLVR_NUM,MC_NUM,KINDER_NUM,KIDS_NUM,GYM_NUM,GOLF_NUM,ACADEMY_NUM,ANIMAL_HSPT_NUM,CAFE,CAR_SHR_NUM,CHILD_MED_NUM,CON_NUM,DPTM_NUM
km_cluster,,,,,,,,,,,,,,,,,,
0,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60
1,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6
2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
3,315,315,315,315,315,315,315,315,315,315,315,315,315,315,315,315,315,315
4,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6
5,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
6,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19
7,42,42,42,42,42,42,42,42,42,42,42,42,42,42,42,42,42,42
8,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6


In [59]:
why = tmp_df[tmp_df['km_cluster']==3]
why

,GU,DONG_CODE,DONG,SUBWAY_NUM,STARBUCKS_NUM,SAFE_DLVR_NUM,MC_NUM,KINDER_NUM,KIDS_NUM,GYM_NUM,GOLF_NUM,ACADEMY_NUM,ANIMAL_HSPT_NUM,CAFE,CAR_SHR_NUM,CHILD_MED_NUM,CON_NUM,DPTM_NUM,km_cluster
index,,,,,,,,,,,,,,,,,,,
0,종로구,1111010100,청운동,0,0,0,0,0,0,0,0,2,1,8,0,0,1,0,3
1,종로구,1111010200,신교동,0,0,0,0,0,0,0,0,5,1,1,0,0,0,0,3
2,종로구,1111010300,궁정동,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,3
3,종로구,1111010400,효자동,0,0,0,0,0,0,0,0,0,1,5,0,0,0,0,3
4,종로구,1111010500,창성동,0,0,0,0,0,0,0,0,1,1,3,0,2,1,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
428,서초구,1165010900,내곡동,0,0,0,0,20,0,0,0,18,1,6,0,11,1,0,3
429,서초구,1165011000,염곡동,0,0,0,0,0,0,0,0,1,1,6,0,3,0,0,3
430,서초구,1165011100,신원동,0,1,0,0,10,0,0,0,7,1,16,0,8,6,0,3


In [60]:
why.describe() # 동물 병원의 수가 1임

,DONG_CODE,SUBWAY_NUM,STARBUCKS_NUM,SAFE_DLVR_NUM,MC_NUM,KINDER_NUM,KIDS_NUM,GYM_NUM,GOLF_NUM,ACADEMY_NUM,ANIMAL_HSPT_NUM,CAFE,CAR_SHR_NUM,CHILD_MED_NUM,CON_NUM,DPTM_NUM,km_cluster
count,3.150000e+02,315.000000,315.000000,315.000000,315.00000,315.000000,315.000000,315.000000,315.000000,315.000000,315.0,315.000000,315.000000,315.000000,315.000000,315.000000,315.0
mean,1.124146e+09,0.187302,0.469841,0.114286,0.04127,1.809524,0.060317,1.069841,0.438095,5.574603,1.0,10.695238,0.923810,3.079365,2.746032,0.028571,3.0
std,1.646594e+07,0.529238,0.849247,0.365231,0.19923,5.005911,0.318506,1.684004,0.877066,7.867331,0.0,9.988328,1.421145,5.053540,3.305893,0.216684,0.0
min,1.111010e+09,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,3.0
25%,1.111018e+09,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,4.000000,0.000000,0.000000,0.000000,0.000000,3.0
50%,1.114017e+09,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,2.000000,1.0,8.000000,0.000000,1.000000,2.000000,0.000000,3.0
75%,1.129013e+09,0.000000,1.000000,0.000000,0.00000,0.000000,0.000000,1.000000,1.000000,8.000000,1.0,15.000000,1.000000,4.000000,3.500000,0.000000,3.0
max,1.168011e+09,3.000000,5.000000,2.000000,1.00000,40.000000,4.000000,11.000000,5.000000,43.000000,1.0,61.000000,7.000000,29.000000,23.000000,3.000000,3.0


In [67]:
col_3 = why.columns[3:-1]
data_3 = why[col_3].values

In [68]:
kmeans_3 = KMeans(n_clusters=5, init='k-means++',max_iter=300,random_state=0)
kmeans_3.fit(data_3)

KMeans(n_clusters=5, random_state=0)

In [69]:
kmeans_3.labels_
tmp_df_3 = why.copy()
tmp_df_3['km_cluster_3'] = kmeans_3.labels_
tmp_df_3.groupby('km_cluster_3').count()

,GU,DONG_CODE,DONG,SUBWAY_NUM,STARBUCKS_NUM,SAFE_DLVR_NUM,MC_NUM,KINDER_NUM,KIDS_NUM,GYM_NUM,GOLF_NUM,ACADEMY_NUM,ANIMAL_HSPT_NUM,CAFE,CAR_SHR_NUM,CHILD_MED_NUM,CON_NUM,DPTM_NUM,km_cluster
km_cluster_3,,,,,,,,,,,,,,,,,,,
0,37,37,37,37,37,37,37,37,37,37,37,37,37,37,37,37,37,37,37
1,31,31,31,31,31,31,31,31,31,31,31,31,31,31,31,31,31,31,31
2,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9
3,58,58,58,58,58,58,58,58,58,58,58,58,58,58,58,58,58,58,58
4,180,180,180,180,180,180,180,180,180,180,180,180,180,180,180,180,180,180,180


In [72]:
whywhy = tmp_df_3[tmp_df_3['km_cluster_3']==4]

In [76]:
whywhy.describe().loc[['mean','min','max']]

,DONG_CODE,SUBWAY_NUM,STARBUCKS_NUM,SAFE_DLVR_NUM,MC_NUM,KINDER_NUM,KIDS_NUM,GYM_NUM,GOLF_NUM,ACADEMY_NUM,ANIMAL_HSPT_NUM,CAFE,CAR_SHR_NUM,CHILD_MED_NUM,CON_NUM,DPTM_NUM,km_cluster,km_cluster_3
mean,1.121838e+09,0.077778,0.172222,0.05,0.016667,0.833333,0.022222,0.461111,0.194444,1.883333,1.0,5.272222,0.461111,0.972222,1.361111,0.005556,3.0,4.0
min,1.111010e+09,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,3.0,4.0
max,1.168011e+09,3.000000,2.000000,1.00,1.000000,10.000000,1.000000,7.000000,2.000000,9.000000,1.0,12.000000,4.000000,16.000000,7.000000,1.000000,3.0,4.0
